In [42]:
import pysam
import collections

import matplotlib.pyplot as plt

In [43]:
#!wget http://krishna.gs.washington.edu/content/members/mouse_ATAC_atlas_website/bams/Cerebellum_62216.bam
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=179PeEpyftCQ3tKhPaRwnVrew-JL8Jn7y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=179PeEpyftCQ3tKhPaRwnVrew-JL8Jn7y" -O Cerebellum_62216.bam && rm -rf /tmp/cookies.txt

--2021-02-21 16:27:26--  https://docs.google.com/uc?export=download&confirm=6k9S&id=179PeEpyftCQ3tKhPaRwnVrew-JL8Jn7y
Resolving docs.google.com (docs.google.com)... 142.250.64.110, 2607:f8b0:4006:807::200e
Connecting to docs.google.com (docs.google.com)|142.250.64.110|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-10-docs.googleusercontent.com/docs/securesc/1bjvcnh7e3n9snqtrasp57c42vjdgsaq/qf2d8o8ce17374ld8f8jtu8pj421e7vm/1613942775000/15445346426572989241/09238621097390475932Z/179PeEpyftCQ3tKhPaRwnVrew-JL8Jn7y?e=download [following]
--2021-02-21 16:27:26--  https://doc-0o-10-docs.googleusercontent.com/docs/securesc/1bjvcnh7e3n9snqtrasp57c42vjdgsaq/qf2d8o8ce17374ld8f8jtu8pj421e7vm/1613942775000/15445346426572989241/09238621097390475932Z/179PeEpyftCQ3tKhPaRwnVrew-JL8Jn7y?e=download
Resolving doc-0o-10-docs.googleusercontent.com (doc-0o-10-docs.googleusercontent.com)... 172.217.165.129, 2607:f8b0:4006:808::2001
Connecting to doc-

In [44]:
bam_file = 'Cerebellum_62216.bam'

In [45]:
sam_file = pysam.Samfile(bam_file, "rb")

In [46]:
insert_sizes = {}

read_names = set()

# Read in the whitelist if provided
barcodes = None

for alignment in sam_file:
    # Don't count R1 and R2 twice
    if alignment.query_name in read_names:
        continue
    else:
        read_names.add(alignment.query_name)

    # Get the insert size
    insert_size = abs(alignment.tlen)

    # Discount inferred insert sizes over 1000
    if insert_size >= 1000:
        continue

    # Track stats
    cell_name = alignment.query_name.split(':')[0]

    if barcodes and cell_name not in barcodes:
        continue

    cell_insert_size_distribution = insert_sizes.get(cell_name, collections.Counter())

    cell_insert_size_distribution[insert_size] += 1
    insert_sizes[cell_name] = cell_insert_size_distribution

In [47]:
cell = list(insert_sizes.keys())[1]

In [48]:
counts = insert_sizes[cell]

In [50]:
plt.bar(counts.keys(), counts.values())

In [23]:
#!samtools view -bs 42.5 Cerebellum_62216.bam > Cerebellum_subsampled.bam

In [14]:
#%load_ext watermark

#%watermark -v -m -p wget,pysam,collections,matplotlib

#print (" ")
#%watermark -u -n -t -z

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.7.3
IPython version      : 7.5.0

wget       : not installed
pysam      : 0.15.2
collections: unknown
matplotlib : 3.0.3

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1062.33.1.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

 
Last updated: Sun Feb 21 2021 15:00:32EST

